# Topic Modeling -- NMF

In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import gensim
import time
import time

import nltk
import stanza
import gensim
import re

#topic modeling
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression as lm

# viz
import matplotlib.collections as mcol
from matplotlib.legend_handler import HandlerLineCollection, HandlerTuple
from matplotlib.lines import Line2D
from normalization2 import *
import matplotlib.pyplot as plt
pd.options.display.max_colwidth=500

import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

#some ipython magic to show the matplotlib plots inline
%matplotlib inline

In [31]:
#df = pd.read_pickle("/home/zz3hs/git/dspg21oss/data/dspg21oss/readme_lemma_try.pkl")

#readme
#df = pd.read_csv("/home/zz3hs/git/dspg21oss/data/dspg21oss/tokenized_readmes.csv")

#repo description
df = pd.read_csv("/home/zz3hs/git/dspg21oss/data/dspg21oss/tokenized_description.csv")

In [32]:
df

,slug,status,tokens
0,davidkpiano/flipping,Done,"flipping,awesome,animations"
1,dotpulse/Dotpulse.OutdatedBrowser,Done,"outdated,browser,neos,cms"
2,victronenergy/dbus-adc,Done,"bridge,venus,device,onboard,adc,dbus"
3,LODIFMO/lodGlossary,Done,"glossary,terms,ipm,deprecated"
4,jacobgumpert/MyArduinoCpp,Done,"simple,test,arduino,repositories"
...,...,...,...
53889,zekunyan/TTGSnackbar,Done,"ttgsnackbar,simple,message,action,button,botto..."
53890,JessYanCoding/MVPArt,Done,"android,mvp,architecture,此,框架,旨,在,解决,传统,mvp,类,..."
53891,cltl/multilingual-wiki-event-pipeline,Done,"project,aims,extract,information,incidents,typ..."
53892,mokha/verdd,Done,"veʹrdd,source,dictionary,editing,framework,foc..."


In [33]:
df["tokens"][1]

'outdated,browser,neos,cms'

In [34]:
# function slightly modified from https://nlpforhackers.io/topic-modeling/

def print_topics(topic_term_mat, vectorizer, top_n=10):
    for idx, topic in enumerate(topic_term_mat):  # loop through each row of H.  idx = row index.  topic = actual row
        print("\nTopic %d:" % (idx))
        #print([(vectorizer.get_feature_names()[i], topic[i])  # printing out words corresponding to indices found in next line
                        #for i in topic.argsort()[:-top_n - 1:-1]])  # finding indices of top words in topic
            
        print_list = [(vectorizer.get_feature_names()[i], topic[i])  
                        for i in topic.argsort()[:-top_n - 1:-1]]
        for item in print_list:
            print(item)
            
# Function to format topics as a "list of list of strings".
# Needed for topic coherence function in Gensim

# function modified from https://nlpforhackers.io/topic-modeling/

def str_topics(topic_term_mat, vectorizer, top_n=10):

    #input. top_n: how many words to list per topic.  If -1, then list all words.
       
    topic_words = []
    
    for idx, topic in enumerate(topic_term_mat):  # loop through each row of H.  idx = row index.  topic = actual row
            
        if top_n == -1:   
            topic_words.append([vectorizer.get_feature_names()[i] for i in topic.argsort()[::-1]])
        else:
            topic_words.append([vectorizer.get_feature_names()[i] for i in topic.argsort()[:-top_n - 1:-1]])
    
    str_wds = []
    
    for wds in topic_words:
        str_wds.append(", ".join(wds))    
    
    return str_wds 

In [35]:
# input needed for LDA, NMF (all from Scikit-Learn) is one string per document (not a list of strings)
text = []
abstract_tokens = df["tokens"]

for tokens in abstract_tokens:
    text.append(tokens)

In [36]:
# vectorize the corpus
tfidf_vectorizer = TfidfVectorizer(max_df=1.0, min_df=3, lowercase=False)
# calculate the feature matrix
tf_idf = tfidf_vectorizer.fit_transform(text)


In [37]:
shape = tf_idf.shape

print("There are", shape[1], "unique tokens in the corpus of",shape[0], "descriptions.")

There are 10649 unique tokens in the corpus of 53894 readmes.


In [38]:
# create model

num_topics = 10

t1 = time.time()
nmf_model = NMF(n_components=num_topics, random_state = 1) #random_state is like a seed in R, control the random number generator
doc_topic = nmf_model.fit_transform(tf_idf)
t2 = time.time()
print(f"  Model run time: {t2-t1}")

topic_term = nmf_model.components_

  Model run time: 4.173003673553467


/home/zz3hs/.conda/envs/crystal_bert/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1076: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


In [39]:
nmf_model

NMF(n_components=10, random_state=1)

In [40]:
topic_frame = pd.DataFrame(doc_topic, columns=["Topic"+" "+str(i) for i in range(num_topics)])
topic_frame

,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8,Topic 9
0,0.000042,0.000054,0.000147,0.000000,0.000174,0.000217,0.000261,0.000000,0.000133,0.001006
1,0.000000,0.000257,0.000087,0.000061,0.000415,0.000897,0.000616,0.000000,0.000000,0.004919
2,0.000000,0.000000,0.000000,0.000000,0.000031,0.000019,0.000548,0.000028,0.000489,0.000968
3,0.000000,0.000079,0.000242,0.000191,0.000070,0.000000,0.000371,0.000000,0.000153,0.001172
4,0.000000,0.000000,0.000044,0.000000,0.000000,0.000000,0.000000,0.000000,0.001592,0.043045
...,...,...,...,...,...,...,...,...,...,...
53889,0.003053,0.000000,0.000000,0.000000,0.000126,0.000000,0.002698,0.000000,0.009597,0.022248
53890,0.001606,0.000000,0.000184,0.000073,0.000036,0.000003,0.001625,0.000000,0.009410,0.000176
53891,0.000000,0.000195,0.015977,0.000000,0.000000,0.000399,0.000647,0.000000,0.000197,0.010804
53892,0.000013,0.000757,0.000000,0.000000,0.000286,0.003865,0.000000,0.000705,0.002301,0.024008


In [41]:
topic_words = str_topics(topic_term, tfidf_vectorizer, top_n=10)
topic_words

['code, com, google, exported, automatically, frozenbubbleandroid, tools, support, android, wordsearch',
 'website, official, source, code, com, repository, academic, personal, public, hugo',
 'project, final, udacity, driving, repository, nanodegree, car, capstone, traffic, software',
 'api, client, wrapper, rest, ruby, server, php, node, sdk, java',
 'blog, jekyll, theme, hugo, pages, source, gatsby, code, io, posts',
 'web, application, de, site, framework, based, page, interface, service, development',
 'library, javascript, java, php, client, ruby, android, net, common, utility',
 'personal, site, portfolio, page, dotfiles, webpage, repository, projects, source, homepage',
 'app, android, react, ios, native, django, mobile, rails, simple, built',
 'python, simple, game, data, framework, based, js, implementation, server, tool']